# Export Pegasus Model to pb Format

## Place this file inside pegasus [folder](https://github.com/google-research/pegasus) 

In [ ]:
import itertools
import os
import time

from absl import logging
from pegasus.data import infeed
from pegasus.params import all_params  # pylint: disable=unused-import
from pegasus.params import estimator_utils
from pegasus.params import registry
import tensorflow as tf
from pegasus.eval import text_eval
from pegasus.ops import public_parsing_ops
import pandas as pd
from random import choice
from tensorflow.python.estimator.export import export
tf.enable_eager_execution()
# import tensorflow_transform as tft
data_name = 'newsroom'
import tensorflow_transform as tft

master = ""
model_dir = "./ckpt/pegasus_ckpt/%s"%data_name
use_tpu = False
iterations_per_loop = 1000
num_shards = 1
param_overrides = "vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6"

eval_dir = os.path.dirname(model_dir)
checkpoint_path = model_dir
checkpoint_path = tf.train.latest_checkpoint(checkpoint_path )
params = registry.get_params('%s_transformer'%data_name)(param_overrides)
pattern = params.dev_pattern
input_fn = infeed.get_input_fn(params.parser, pattern,
                                     tf.estimator.ModeKeys.PREDICT)
parser, shapes = params.parser(mode=tf.estimator.ModeKeys.PREDICT)

In [ ]:
RAW_DATA_FEATURE_SPEC = dict([("inputs", tf.io.FixedLenFeature(shapes['inputs'], tf.int64)), 
                               ('targets', tf.io.FixedLenFeature(shapes['targets'], tf.string))])

raw_feature_spec = RAW_DATA_FEATURE_SPEC.copy()
raw_feature_spec.pop('targets')
# raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#         raw_feature_spec, default_batch_size=0)
def serving_input_fn():
    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=1)
    serving_input_receiver = raw_input_fn()

    raw_features = serving_input_receiver.features
    return tf.estimator.export.ServingInputReceiver(
        raw_features, serving_input_receiver.receiver_tensors)

print(tf.executing_eagerly())
estimator = estimator_utils.create_estimator(master, 
                                             model_dir,
                                             use_tpu,
                                             iterations_per_loop,
                                             num_shards, params, include_features_in_predictions=False)

estimator.export_saved_model(
    "model/", serving_input_fn
)